# Preparing the data for the model training

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from openpyxl import Workbook
from sklearn.linear_model import LinearRegression

# Assuming these functions are defined in your local modules
from _managing_data import update_data
from common_functions import calculate_heikin_ashi, calculate_supertrend, calculate_macd, calculate_indicators, macd_signals

In [2]:
# Define symbols and timeframes
# timeframes = ['30m', '1h']
# symbols = ['BTC/USDT', 'ETH/USDT']

symbols = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'SOL/USDT', 'XRP/USDT', 'ADA/USDT', 'DOGE/USDT',
               'DOT/USDT', 'LINK/USDT', 'IMX/USDT', 'ICP/USDT']
timeframes = ['15m', '30m', '1h', '2h', '4h']

In [3]:
# Manual mappings for categorical features
symbol_mapping = {
    'BTC/USDT': 0, 'ETH/USDT': 1, 'BNB/USDT': 2, 'SOL/USDT': 3, 'XRP/USDT': 4,
    'ADA/USDT': 5, 'DOGE/USDT': 6, 'DOT/USDT': 7, 'LINK/USDT': 8, 'IMX/USDT': 9, 'ICP/USDT': 10
}
timeframe_mapping = {
    '15m': 0, '30m': 1, '1h': 2, '2h': 3, '4h': 4
}
type_mapping = {
    'Long': 0, 'Short': 1
}

In [4]:
higher_timeframes = {
    '15m': '1h',
    '30m': '1h',
    '1h': '4h',
    '2h': '4h',
    '4h': '1d'
}

In [5]:
def get_linear_coeffs(values):
    """Calculate linear regression coefficients k (slope) and b (intercept)."""

    x = np.arange(1, 6)  # x values from 1 to 5
    y = np.array(values)

    # Calculate the coefficients of the linear fit (slope and intercept)
    slope, intercept = np.polyfit(x, y, 1)
    return slope, intercept

In [6]:
def get_previous_values(df_higher, signal_time, column):
    """Retrieve the last five values of the specified column just before the signal time from the higher timeframe data."""
    valid_times = df_higher[df_higher.index <= signal_time]
    if not valid_times.empty:
        last_times = valid_times.index[-5:]  # Get the last 5 times
        return df_higher.loc[last_times][column].values, last_times
    return None, None

In [7]:
def save_results_to_excel(results, filename='backtesting_results.xlsx'):
    df = pd.DataFrame(results)
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Consolidated Results', index=False)

In [8]:
def backtest_strategy(df, df_higher):
    initial_capital = 10000
    risk_per_trade = 0.05  # 5% of capital risked per trade
    results = []
    capital = initial_capital
    max_drawdown = 0
    peak_capital = capital
    last_exit_time = None  # This will keep track of the last exit time

    tp_multipliers = [1, 1.5, 2]

    for index, row in df.iterrows():
        # Skip to next iteration if current row time is before last exit time
        if last_exit_time is not None and row.name <= last_exit_time:
            continue

        # Check for signals, but open position on the next candle
        signal = 0
        if row['signal'] == 1:  # Long signal
            next_index = df.index.get_loc(index) + 1
            if next_index >= len(df):
                continue  # Skip if there is no next row
            next_row = df.iloc[next_index]
            # Verify SuperTrend and SMA conditions on the next candle
            if next_row['HA_close'] > next_row['SuperTrend'] and next_row['HA_close'] > next_row['SMA200']:
                signal = 1  # Open long position on next candle

        elif row['signal'] == -1:  # Short signal
            next_index = df.index.get_loc(index) + 1
            if next_index >= len(df):
                continue  # Skip if there is no next row
            next_row = df.iloc[next_index]
            # Verify SuperTrend and SMA conditions on the next candle
            if next_row['HA_close'] < next_row['SuperTrend'] and next_row['HA_close'] < next_row['SMA200']:
                signal = -1  # Open short position on next candle

        if signal == 0:
            continue  # No valid signal, skip the iteration

        # Now we proceed with trade execution
        entry_price = next_row['open']  # Use the next candle's open price
        atr = next_row['ATR']
        super_trend = next_row['SuperTrend']
        stop_loss = super_trend - atr if signal == 1 else super_trend + atr
        stop_loss_distance = abs(entry_price - stop_loss)
        position_size = (risk_per_trade * capital) / stop_loss_distance

        # Fetch the last five RSI, MACD, and Volume values from the higher timeframe
        previous_rsi_values, rsi_times = get_previous_values(df_higher, next_row.name, 'RSI')
        previous_macd_values, macd_times = get_previous_values(df_higher, next_row.name, 'macd')
        previous_volume_values, volume_times = get_previous_values(df_higher, next_row.name, 'volume')

        rsi_k, rsi_b = get_linear_coeffs(previous_rsi_values) if previous_rsi_values is not None else (None, None)
        macd_k, macd_b = get_linear_coeffs(previous_macd_values) if previous_macd_values is not None else (None, None)
        volume_k, volume_b = get_linear_coeffs(previous_volume_values) if previous_volume_values is not None else (None, None)

        future_rows = df.iloc[next_index + 1:]  # Start checking from the row after the entry candle

        for tp_multiplier in tp_multipliers:
            tp = entry_price + tp_multiplier * stop_loss_distance if signal == 1 else entry_price - tp_multiplier * stop_loss_distance
            exit_price = None
            highest_high = next_row['high']
            lowest_low = next_row['low']

            for j, future_row in future_rows.iterrows():
                if signal == 1:
                    highest_high = max(highest_high, future_row['high'])
                    if future_row['low'] <= stop_loss or future_row['high'] >= tp:
                        exit_price = stop_loss if future_row['low'] <= stop_loss else tp
                        break
                else:
                    lowest_low = min(lowest_low, future_row['low'])
                    if future_row['high'] >= stop_loss or future_row['low'] <= tp:
                        exit_price = stop_loss if future_row['high'] >= stop_loss else tp
                        break

            if exit_price is None:
                exit_price = next_row['close']  # Default exit if no stop/TP was hit

            last_exit_time = future_row.name  # Update last exit time with the end of the current trade

            optimum_closing = tp if exit_price == tp else (highest_high if signal == 1 else lowest_low)
            profit = (exit_price - entry_price) * position_size if signal == 1 else (entry_price - exit_price) * position_size

            # Record the previous RSI and MACD values directly in the results
            results.append({
                'Symbol': next_row['Symbol'], 'Timeframe': next_row['Timeframe'],
                'Entry Price': entry_price, 'Exit Price': exit_price,
                'Profit': profit, 'Type': 'Long' if signal == 1 else 'Short',
                'Entry Date': next_row.name, 'Exit Date': future_row.name, 'TP Multiplier': tp_multiplier,
                'Optimum Closing': optimum_closing,
                'Previous RSI Values': previous_rsi_values.tolist() if previous_rsi_values is not None else [],
                'Previous MACD Values': previous_macd_values.tolist() if previous_macd_values is not None else [],
                'RSI Line Slope (k)': rsi_k, 'RSI Line Intercept (b)': rsi_b,
                'MACD Line Slope (k)': macd_k, 'MACD Line Intercept (b)': macd_b,
                'Volume Line Slope (k)': volume_k, 'Volume Line Intercept (b)': volume_b
            })

    return results

In [9]:
def main():
    results = []
    for symbol in symbols:
        for timeframe in timeframes:
            print(f"Processing {symbol} on {timeframe} timeframe")
            df = update_data(symbol, timeframe)  # Fetch data
            df = calculate_heikin_ashi(df)  # Convert data to Heikin Ashi
            df = calculate_indicators(df)  # Calculate indicators like MACD, ATR, etc.
            df = macd_signals(df)  # Identify MACD buy/sell signals
            df['Symbol'] = symbol  # Add symbol column for reference
            df['Timeframe'] = timeframe  # Add timeframe column for reference

            df_higher = update_data(symbol, higher_timeframes[timeframe])
            df_higher = calculate_heikin_ashi(df_higher)  # Convert higher timeframe data to Heikin Ashi
            df_higher = calculate_indicators(df_higher)  # Calculate indicators for the higher timeframe

            trades = backtest_strategy(df, df_higher)  # Perform backtesting
            results.extend(trades)  # Collect all trades across symbols and timeframes

    save_results_to_excel(results, filename='backtesting_results.xlsx')
    print("Backtesting completed and results are saved.")

if __name__ == "__main__":
    main()


Processing BTC/USDT on 15m timeframe


2024-09-28 19:01:40,710 - ERROR - Error fetching data for BTC/USDT on 15m: binance GET https://api.binance.com/api/v3/exchangeInfo
2024-09-28 19:01:47,818 - INFO - Data for BTC/USDT on 15m saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.

Processing BTC/USDT on 30m timeframe


2024-09-28 19:02:50,535 - ERROR - Error fetching data for BTC/USDT on 30m: binance GET https://api.binance.com/api/v3/exchangeInfo
2024-09-28 19:02:54,024 - INFO - Data for BTC/USDT on 30m saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.

Processing BTC/USDT on 1h timeframe


...

Processing ICP/USDT on 2h timeframe


2024-09-28 19:30:15,718 - INFO - Data for ICP/USDT on 2h saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['HA_open'].iloc[0] = (data['open'].ilo

Processing ICP/USDT on 4h timeframe


2024-09-28 19:30:30,775 - INFO - Data for ICP/USDT on 4h saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['HA_open'].iloc[0] = (data['open'].ilo

Backtesting completed and results are saved.


# ML-training

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

In [11]:
# Load data from Excel
df = pd.read_excel('backtesting_results.xlsx')

In [12]:
# Apply mappings
df['Symbol'] = df['Symbol'].map(symbol_mapping)
df['Timeframe'] = df['Timeframe'].map(timeframe_mapping)
df['Type'] = df['Type'].map(type_mapping)

In [13]:
# Selecting the necessary columns
features = ['Symbol', 'Timeframe', 'Entry Price', 'Type', 'TP Multiplier',
            'RSI Line Slope (k)', 'RSI Line Intercept (b)', 'MACD Line Slope (k)', 'MACD Line Intercept (b)',
            'Volume Line Slope (k)', 'Volume Line Intercept (b)']
X = df[features]

In [14]:
# Normalizing continuous data
continuous_features = ['Entry Price', 'TP Multiplier', 'RSI Line Slope (k)', 'RSI Line Intercept (b)',
                       'MACD Line Slope (k)', 'MACD Line Intercept (b)',
                       'Volume Line Slope (k)', 'Volume Line Intercept (b)']
scaler = StandardScaler()
X[continuous_features] = scaler.fit_transform(X[continuous_features])

C:\Users\Asus\AppData\Local\Temp\ipykernel_12016\1080446843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[continuous_features] = scaler.fit_transform(X[continuous_features])


In [15]:
X

,Symbol,Timeframe,Entry Price,Type,TP Multiplier,RSI Line Slope (k),RSI Line Intercept (b),MACD Line Slope (k),MACD Line Intercept (b),Volume Line Slope (k),Volume Line Intercept (b)
0,0,0,1.235204,0,-1.224745,0.637669,0.116343,0.617581,0.112179,-0.064584,-0.087381
1,0,0,1.235204,0,0.000000,0.637669,0.116343,0.617581,0.112179,-0.064584,-0.087381
2,0,0,1.235204,0,1.224745,0.637669,0.116343,0.617581,0.112179,-0.064584,-0.087381
3,0,0,1.237517,0,-1.224745,0.658586,-0.155058,-0.393807,0.394731,-0.064351,-0.087429
4,0,0,1.237517,0,0.000000,0.658586,-0.155058,-0.393807,0.394731,-0.064351,-0.087429
...,...,...,...,...,...,...,...,...,...,...,...
28480,10,4,-0.312671,1,0.000000,-1.280958,0.014578,-0.019335,-0.022001,-0.058173,-0.079706
28481,10,4,-0.312671,1,1.224745,-1.280958,0.014578,-0.019335,-0.022001,-0.058173,-0.079706
28482,10,4,-0.312580,0,-1.224745,1.831756,0.118310,-0.009365,-0.023710,-0.058515,-0.072757
28483,10,4,-0.312580,0,0.000000,1.831756,0.118310,-0.009365,-0.023710,-0.058515,-0.072757


In [16]:
# Calculate Profit Change
df['Profit change'] = df['Entry Price'] / df['Optimum Closing']
y = df['Profit change']

In [17]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Gradient Boosting Regressor
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [18]:
# Predict on the testing set
y_pred = model.predict(X_test)

print(X_test)

# Calculate and print the performance metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save the trained model for later use
joblib.dump(model, 'gradient_boosting_regressor.pkl')
print("Model saved successfully!")

       Symbol  Timeframe  Entry Price  Type  TP Multiplier  \
4596        1          1    -0.136063     1      -1.224745   
25405       9          2    -0.313196     0       0.000000   
8957        3          1    -0.311369     0       1.224745   
8870        3          1    -0.310730     1       1.224745   
96          0          0     1.021946     1      -1.224745   
...       ...        ...          ...   ...            ...   
2826        1          0    -0.213019     1      -1.224745   
28372      10          3    -0.312592     0       0.000000   
20887       8          0    -0.312819     1       0.000000   
5673        2          0    -0.290732     1      -1.224745   
5262        1          3    -0.163853     1      -1.224745   

       RSI Line Slope (k)  RSI Line Intercept (b)  MACD Line Slope (k)  \
4596            -1.026709                0.510688            -0.001644   
25405            0.057947                0.805700            -0.013380   
8957             1.045838        

# Backtesting

# Live prediction

In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from _managing_data import update_data
from common_functions import calculate_heikin_ashi, calculate_indicators, macd_signals

In [58]:
symbol = 'BTC/USDT'
timeframe = '1h'
tp_multiplier = 1.5
df = update_data(symbol, timeframe)  # Fetch data
df = calculate_heikin_ashi(df)  # Convert to Heikin Ashi
df = calculate_indicators(df)

2024-09-28 17:37:56,509 - INFO - Data for BTC/USDT on 1h saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['HA_open'].iloc[0] = (data['open'].ilo

In [20]:
# Updated get_signals function
def get_signals(df):
    df['signal'] = 0  # Initialize signal column
    last_signal_date = None  # Variable to store the date of the last valid signal
    last_signal_type = None  # Variable to store the type of the last valid signal

    for i in range(1, len(df) - 2):
        # Detect MACD crossover for long positions
        if df['macd'].iloc[i] > df['macd_signal'].iloc[i] and df['macd'].iloc[i - 1] <= df['macd_signal'].iloc[i - 1]:
            # Checking SMA and SuperTrend confirmation
            if df['HA_close'].iloc[i + 1] > df['SuperTrend'].iloc[i + 1] and df['HA_close'].iloc[i + 1] > df['SMA200'].iloc[i + 1]:
                df.loc[df.index[i + 2], 'signal'] = 1  # Long signal
                last_signal_date = df.index[i + 1]  # Update last signal date
                last_signal_type = 'Long'  # Update last signal type

        # Detect MACD crossover for short positions
        elif df['macd'].iloc[i] < df['macd_signal'].iloc[i] and df['macd'].iloc[i - 1] >= df['macd_signal'].iloc[i - 1]:
            # Checking SMA and SuperTrend confirmation
            if df['HA_close'].iloc[i + 1] < df['SuperTrend'].iloc[i + 1] and df['HA_close'].iloc[i + 1] < df['SMA200'].iloc[i + 1]:
                df.loc[df.index[i + 2], 'signal'] = -1  # Short signal
                last_signal_date = df.index[i + 1]  # Update last signal date
                last_signal_type = 'Short'  # Update last signal type

    return last_signal_date, last_signal_type  # Return the date and type of the last valid signal

In [60]:
# Define the function to retrieve the last five values before the signal time
def get_previous_values(df_higher, signal_time, column):
    """Retrieve the last five values of the specified column just before the signal time from the higher timeframe data."""
    valid_times = df_higher[df_higher.index <= signal_time]
    if not valid_times.empty:
        last_times = valid_times.index[-5:]  # Get the last 5 times
        return df_higher.loc[last_times][column].values, last_times
    return None, None

# Define the function to calculate linear regression coefficients
def get_linear_coeffs(values):
    """Calculate linear regression coefficients k (slope) and b (intercept)."""
    x = np.arange(1, 6)  # x values from 1 to 5
    y = np.array(values)

    # Calculate the coefficients of the linear fit (slope and intercept)
    slope, intercept = np.polyfit(x, y, 1)
    return slope, intercept

# Define the higher timeframes mapping
higher_timeframes = {
    '15m': '1h',
    '30m': '1h',
    '1h': '4h',
    '2h': '4h',
    '4h': '1d'
}

In [61]:
# Preparing higher timeframe
df_higher = update_data(symbol, higher_timeframes[timeframe])
df_higher = calculate_heikin_ashi(df_higher)  # Convert to Heikin Ashi
df_higher = calculate_indicators(df_higher)

2024-09-28 17:38:02,123 - INFO - Data for BTC/USDT on 4h saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['HA_open'].iloc[0] = (data['open'].ilo

In [62]:
last_signal_date, last_signal_type = get_signals(df)

In [63]:
# Manual mappings for categorical features
symbol_mapping = {
    'BTC/USDT': 0, 'ETH/USDT': 1, 'BNB/USDT': 2, 'SOL/USDT': 3, 'XRP/USDT': 4,
    'ADA/USDT': 5, 'DOGE/USDT': 6, 'DOT/USDT': 7, 'LINK/USDT': 8, 'IMX/USDT': 9, 'ICP/USDT': 10
}
timeframe_mapping = {
    '15m': 0, '30m': 1, '1h': 2, '2h': 3, '4h': 4
}
type_mapping = {
    'Long': 0, 'Short': 1
}

In [64]:
# get_previous_values(df_higher, last_signal_date, 'RSI')[0]
previous_rsi_values, rsi_times = get_previous_values(df_higher, last_signal_date, 'RSI')
previous_macd_values, macd_times = get_previous_values(df_higher, last_signal_date, 'macd')
previous_volume_values, volume_times = get_previous_values(df_higher, last_signal_date, 'volume')

rsi_k, rsi_b = get_linear_coeffs(previous_rsi_values) if previous_rsi_values is not None else (None, None)
macd_k, macd_b = get_linear_coeffs(previous_macd_values) if previous_macd_values is not None else (None, None)
volume_k, volume_b = get_linear_coeffs(previous_volume_values) if previous_volume_values is not None else (None, None)


# Create a dictionary with the specified data
data = {
    'Symbol': symbol,
    'Timeframe': timeframe,
    'Entry Price': df.loc[last_signal_date, 'HA_open'],
    'Type': last_signal_type,
    'TP Multiplier': tp_multiplier,
    'RSI Line Slope (k)': rsi_k,
    'RSI Line Intercept (b)': rsi_b,
    'MACD Line Slope (k)': macd_k,
    'MACD Line Intercept (b)': macd_b,
    'Volume Line Slope (k)': volume_k,
    'Volume Line Intercept (b)': volume_b
}

# Convert the dictionary to a pandas DataFrame
data_df = pd.DataFrame([data])  # Wrap data in a list to create a single-row DataFrame



# Apply mappings
data_df['Symbol'] = data_df['Symbol'].map(symbol_mapping)
data_df['Timeframe'] = data_df['Timeframe'].map(timeframe_mapping)
data_df['Type'] = data_df['Type'].map(type_mapping)


# Normalizing continuous data
continuous_features = ['Entry Price', 'TP Multiplier', 'RSI Line Slope (k)', 'RSI Line Intercept (b)',
                       'MACD Line Slope (k)', 'MACD Line Intercept (b)',
                       'Volume Line Slope (k)', 'Volume Line Intercept (b)']
scaler = StandardScaler()
data_df[continuous_features] = scaler.fit_transform(data_df[continuous_features])


# # Display the DataFrame
# data_df


In [65]:
# Make the prediction
model = joblib.load('gradient_boosting_regressor.pkl')
predictions = model.predict(data_df)

# Use the predictin

Optimum_Closing = df.loc[last_signal_date, 'HA_open'] / predictions

# print(predictions)
print(Optimum_Closing)

[68176.64965841]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from _managing_data import update_data
from common_functions import calculate_heikin_ashi, calculate_indicators, macd_signals

symbol = 'BTC/USDT'
timeframe = '1h'
tp_multiplier = 1.5
df = update_data(symbol, timeframe)  # Fetch data
df = calculate_heikin_ashi(df)  # Convert to Heikin Ashi
df = calculate_indicators(df)

# Updated get_signals function
def get_signals(df):
    df['signal'] = 0  # Initialize signal column
    last_signal_date = None  # Variable to store the date of the last valid signal
    last_signal_type = None  # Variable to store the type of the last valid signal

    for i in range(1, len(df) - 2):
        # Detect MACD crossover for long positions
        if df['macd'].iloc[i] > df['macd_signal'].iloc[i] and df['macd'].iloc[i - 1] <= df['macd_signal'].iloc[i - 1]:
            # Checking SMA and SuperTrend confirmation
            if df['HA_close'].iloc[i + 1] > df['SuperTrend'].iloc[i + 1] and df['HA_close'].iloc[i + 1] > df['SMA200'].iloc[i + 1]:
                df.loc[df.index[i + 2], 'signal'] = 1  # Long signal
                last_signal_date = df.index[i + 1]  # Update last signal date
                last_signal_type = 'Long'  # Update last signal type

        # Detect MACD crossover for short positions
        elif df['macd'].iloc[i] < df['macd_signal'].iloc[i] and df['macd'].iloc[i - 1] >= df['macd_signal'].iloc[i - 1]:
            # Checking SMA and SuperTrend confirmation
            if df['HA_close'].iloc[i + 1] < df['SuperTrend'].iloc[i + 1] and df['HA_close'].iloc[i + 1] < df['SMA200'].iloc[i + 1]:
                df.loc[df.index[i + 2], 'signal'] = -1  # Short signal
                last_signal_date = df.index[i + 1]  # Update last signal date
                last_signal_type = 'Short'  # Update last signal type

    return last_signal_date, last_signal_type  # Return the date and type of the last valid signal

# Define the function to retrieve the last five values before the signal time
def get_previous_values(df_higher, signal_time, column):
    """Retrieve the last five values of the specified column just before the signal time from the higher timeframe data."""
    valid_times = df_higher[df_higher.index <= signal_time]
    if not valid_times.empty:
        last_times = valid_times.index[-5:]  # Get the last 5 times
        return df_higher.loc[last_times][column].values, last_times
    return None, None

# Define the function to calculate linear regression coefficients
def get_linear_coeffs(values):
    """Calculate linear regression coefficients k (slope) and b (intercept)."""
    x = np.arange(1, 6)  # x values from 1 to 5
    y = np.array(values)

    # Calculate the coefficients of the linear fit (slope and intercept)
    slope, intercept = np.polyfit(x, y, 1)
    return slope, intercept

# Define the higher timeframes mapping
higher_timeframes = {
    '15m': '1h',
    '30m': '1h',
    '1h': '4h',
    '2h': '4h',
    '4h': '1d'
}

# Preparing higher timeframe
df_higher = update_data(symbol, higher_timeframes[timeframe])
df_higher = calculate_heikin_ashi(df_higher)  # Convert to Heikin Ashi
df_higher = calculate_indicators(df_higher)

last_signal_date, last_signal_type = get_signals(df)

# Manual mappings for categorical features
symbol_mapping = {
    'BTC/USDT': 0, 'ETH/USDT': 1, 'BNB/USDT': 2, 'SOL/USDT': 3, 'XRP/USDT': 4,
    'ADA/USDT': 5, 'DOGE/USDT': 6, 'DOT/USDT': 7, 'LINK/USDT': 8, 'IMX/USDT': 9, 'ICP/USDT': 10
}
timeframe_mapping = {
    '15m': 0, '30m': 1, '1h': 2, '2h': 3, '4h': 4
}
type_mapping = {
    'Long': 0, 'Short': 1
}

# get_previous_values(df_higher, last_signal_date, 'RSI')[0]
previous_rsi_values, rsi_times = get_previous_values(df_higher, last_signal_date, 'RSI')
previous_macd_values, macd_times = get_previous_values(df_higher, last_signal_date, 'macd')
previous_volume_values, volume_times = get_previous_values(df_higher, last_signal_date, 'volume')

rsi_k, rsi_b = get_linear_coeffs(previous_rsi_values) if previous_rsi_values is not None else (None, None)
macd_k, macd_b = get_linear_coeffs(previous_macd_values) if previous_macd_values is not None else (None, None)
volume_k, volume_b = get_linear_coeffs(previous_volume_values) if previous_volume_values is not None else (None, None)


# Create a dictionary with the specified data
data = {
    'Symbol': symbol,
    'Timeframe': timeframe,
    'Entry Price': df.loc[last_signal_date, 'HA_open'],
    'Type': last_signal_type,
    'TP Multiplier': tp_multiplier,
    'RSI Line Slope (k)': rsi_k,
    'RSI Line Intercept (b)': rsi_b,
    'MACD Line Slope (k)': macd_k,
    'MACD Line Intercept (b)': macd_b,
    'Volume Line Slope (k)': volume_k,
    'Volume Line Intercept (b)': volume_b
}

# Convert the dictionary to a pandas DataFrame
data_df = pd.DataFrame([data])  # Wrap data in a list to create a single-row DataFrame



# Apply mappings
data_df['Symbol'] = data_df['Symbol'].map(symbol_mapping)
data_df['Timeframe'] = data_df['Timeframe'].map(timeframe_mapping)
data_df['Type'] = data_df['Type'].map(type_mapping)


# Normalizing continuous data
continuous_features = ['Entry Price', 'TP Multiplier', 'RSI Line Slope (k)', 'RSI Line Intercept (b)',
                       'MACD Line Slope (k)', 'MACD Line Intercept (b)',
                       'Volume Line Slope (k)', 'Volume Line Intercept (b)']
scaler = StandardScaler()
data_df[continuous_features] = scaler.fit_transform(data_df[continuous_features])


# # Display the DataFrame
# data_df

# Make the prediction
model = joblib.load('gradient_boosting_regressor.pkl')
predictions = model.predict(data_df)

# Use the predictin

Optimum_Closing = df.loc[last_signal_date, 'HA_open'] / predictions

# print(predictions)
print(Optimum_Closing)

In [ ]:
trade_data = [
    {'Symbol': 'BTC/USDT', 'Timeframe': '1h', 'TP Multiplier': 2},
    {'Symbol': 'ETH/USDT', 'Timeframe': '4h', 'TP Multiplier': 1.5},
    # Add more trade data as needed
]


In [27]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from _managing_data import update_data
from common_functions import calculate_heikin_ashi, calculate_indicators, macd_signals

# Define the list of trade data
trade_data = [
    {'Symbol': 'DOGE/USDT', 'Timeframe': '2h', 'TP Multiplier': 2}
    # Add more trade data as needed
]

# Mappings for categorical features
symbol_mapping = {
    'BTC/USDT': 0, 'ETH/USDT': 1, 'BNB/USDT': 2, 'SOL/USDT': 3, 'XRP/USDT': 4,
    'ADA/USDT': 5, 'DOGE/USDT': 6, 'DOT/USDT': 7, 'LINK/USDT': 8, 'IMX/USDT': 9, 'ICP/USDT': 10
}
timeframe_mapping = {
    '15m': 0, '30m': 1, '1h': 2, '2h': 3, '4h': 4
}
type_mapping = {
    'Long': 0, 'Short': 1
}

# Higher timeframe mappings
higher_timeframes = {
    '15m': '1h',
    '30m': '1h',
    '1h': '4h',
    '2h': '4h',
    '4h': '1d'
}

# Load the model
model = joblib.load('gradient_boosting_regressor.pkl')

# Function to fetch, process, and predict for each trade
def process_trade(symbol, timeframe, tp_multiplier):
    # Fetch data for the symbol and timeframe
    df = update_data(symbol, timeframe)
    df = calculate_heikin_ashi(df)
    df = calculate_indicators(df)

    # Fetch data for the higher timeframe
    df_higher = update_data(symbol, higher_timeframes[timeframe])
    df_higher = calculate_heikin_ashi(df_higher)
    df_higher = calculate_indicators(df_higher)

    # Get the last signal date and type
    last_signal_date, last_signal_type = get_signals(df)

    # Retrieve previous values for RSI, MACD, and volume from the higher timeframe
    previous_rsi_values, _ = get_previous_values(df_higher, last_signal_date, 'RSI')
    previous_macd_values, _ = get_previous_values(df_higher, last_signal_date, 'macd')
    previous_volume_values, _ = get_previous_values(df_higher, last_signal_date, 'volume')

    # Calculate linear regression coefficients
    rsi_k, rsi_b = get_linear_coeffs(previous_rsi_values) if previous_rsi_values is not None else (None, None)
    macd_k, macd_b = get_linear_coeffs(previous_macd_values) if previous_macd_values is not None else (None, None)
    volume_k, volume_b = get_linear_coeffs(previous_volume_values) if previous_volume_values is not None else (None, None)

    # Create a dictionary with the data for prediction
    data = {
        'Symbol': symbol,
        'Timeframe': timeframe,
        'Entry Price': df.loc[last_signal_date, 'HA_open'],
        'Type': last_signal_type,
        'TP Multiplier': tp_multiplier,
        'RSI Line Slope (k)': rsi_k,
        'RSI Line Intercept (b)': rsi_b,
        'MACD Line Slope (k)': macd_k,
        'MACD Line Intercept (b)': macd_b,
        'Volume Line Slope (k)': volume_k,
        'Volume Line Intercept (b)': volume_b
    }

    # Convert the dictionary to a DataFrame
    data_df = pd.DataFrame([data])

    # Apply mappings for categorical columns
    data_df['Symbol'] = data_df['Symbol'].map(symbol_mapping)
    data_df['Timeframe'] = data_df['Timeframe'].map(timeframe_mapping)
    data_df['Type'] = data_df['Type'].map(type_mapping)

    # Normalize the continuous features
    continuous_features = ['Entry Price', 'TP Multiplier', 'RSI Line Slope (k)', 'RSI Line Intercept (b)',
                           'MACD Line Slope (k)', 'MACD Line Intercept (b)',
                           'Volume Line Slope (k)', 'Volume Line Intercept (b)']
    scaler = StandardScaler()
    data_df[continuous_features] = scaler.fit_transform(data_df[continuous_features])

    # Make predictions using the model
    predictions = model.predict(data_df)

    # Calculate Optimum Closing
    Optimum_Closing = df.loc[last_signal_date, 'HA_open'] / predictions

    # Return results
    return {
        'Symbol': symbol,
        'Timeframe': timeframe,
        'Type': last_signal_type,
        'Entry Price': df.loc[last_signal_date, 'HA_open'],
        'Optimum Closing': Optimum_Closing[0],  # Assuming it's a single-row result
        'Predicted Profit': predictions[0]
    }

# Process each trade in the trade_data list
results = []
for trade in trade_data:
    result = process_trade(trade['Symbol'], trade['Timeframe'], trade['TP Multiplier'])
    results.append(result)

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Print the results
print(results_df)


2024-09-28 20:03:03,571 - INFO - Data for DOGE/USDT on 2h saved/updated successfully.
E:\side projects\backtesting&visualization\common_functions.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['HA_open'].iloc[0] = (data['open'].il

      Symbol Timeframe  Type  Entry Price  Optimum Closing  Predicted Profit
0  DOGE/USDT        2h  Long       0.1117         0.119734          0.932905


In [26]:
# Print the results
print(results_df)

      Symbol Timeframe   Type  Entry Price  Optimum Closing  Predicted Profit
0  LINK/USDT        1h   Long     12.72500        13.477891          0.944139
1   BTC/USDT        1h   Long  65960.34000     69795.868720          0.945046
2  DOGE/USDT        1h   Long      0.12169         0.128766          0.945046
3   IMX/USDT       30m  Short      1.74400         1.706320          1.022082
4   ADA/USDT        1h   Long      0.40255         0.425958          0.945046
5   IMX/USDT       15m  Short      1.74850         1.723026          1.014785
6   BNB/USDT        4h   Long    608.65000       654.631488          0.929760
7   ICP/USDT        2h   Long      9.63550        10.408253          0.925756
